In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler

In [2]:
y_train = pd.read_csv("meteorological/y_train.csv", index_col=0, parse_dates=True)
y_train.index.freq='5T'
y = pd.read_csv("meteorological/y.csv", index_col=0, parse_dates=True)
y.index.freq = '5T'

In [12]:
n_input_target = [25, 68, 156, 288, 576]

In [13]:
scaler = MinMaxScaler()
scaler.fit(y_train)

MinMaxScaler()

In [14]:
scaled_train = scaler.transform(y_train)
scaled_test = scaler.transform(y)

In [46]:
n_input = n_input_target[0] #순서대로 하나씩 넣어보기
n_features = 1

generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
model = Sequential()
model.add(tf.keras.layers.LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(generator, epochs=1)

99623/99623 [==============================] - 2147s 22ms/step - loss: 8.6014e-05


In [62]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(y)):
    # first_eval_batch = scaled_train[-n_input:]
    # current_batch = first_eval_batch.reshape((1, n_input, n_features))
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = model.predict(current_batch)[0]

    # store prediction
    test_predictions.append(current_pred)

    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

1/1 [==============================] - 0s 12ms/step


In [67]:
first_eval_batch

array([[0.36302967],
       [0.36296172],
       [0.36153497],
       [0.36119466],
       [0.35636684],
       [0.35651822],
       [0.35633966],
       [0.36001026],
       [0.35639365],
       [0.35551917],
       [0.35486724],
       [0.35527267],
       [0.35051318],
       [0.35735626],
       [0.35626039],
       [0.36101856],
       [0.35588195],
       [0.36224391],
       [0.35990588],
       [0.34889957],
       [0.34659551],
       [0.34416877],
       [0.33697518],
       [0.33244691],
       [0.32195814]])

In [63]:
# first_eval_batch = scaled_train[-n_input:]
# current_batch = first_eval_batch.reshape((1, n_input, n_features))
current_batch.shape

(1, 25, 1)

In [64]:
test_predictions

[array([0.31508565], dtype=float32),
 array([0.30733627], dtype=float32),
 array([0.2992134], dtype=float32),
 array([0.29075688], dtype=float32),
 array([0.28216666], dtype=float32),
 array([0.27357882], dtype=float32),
 array([0.26503396], dtype=float32),
 array([0.25656176], dtype=float32),
 array([0.24787754], dtype=float32),
 array([0.23918527], dtype=float32),
 array([0.23044], dtype=float32),
 array([0.22159286], dtype=float32),
 array([0.21259719], dtype=float32),
 array([0.20340613], dtype=float32),
 array([0.19462472], dtype=float32),
 array([0.18627444], dtype=float32),
 array([0.17831524], dtype=float32),
 array([0.17069747], dtype=float32),
 array([0.16385579], dtype=float32),
 array([0.15780425], dtype=float32),
 array([0.15146846], dtype=float32),
 array([0.1449038], dtype=float32),
 array([0.13813037], dtype=float32),
 array([0.13113368], dtype=float32),
 array([0.12387764], dtype=float32),
 array([0.11630875], dtype=float32),
 array([0.10850249], dtype=float32),
 array

In [65]:
scaled_test

array([[0.31916749],
       [0.32242171],
       [0.31781888],
       ...,
       [0.29923902],
       [0.29978998],
       [0.2916434 ]])

## Inverse Transformations and Compare

In [31]:
true_predictions = scaler.inverse_transform(test_predictions)

ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
true_predictions

In [ ]:
y

In [ ]:
# IGNORE WARNINGS
y['Predictions'] = true_predictions

In [ ]:
y

In [ ]:
y.plot(figsize=(12,8))

# Saving and Loading Models

In [ ]:
model.save(f'LSTM{n_input}.h5')

## load a model

In [ ]:
from keras.models import load_model
new_model = load_model(f'LSTM{n_input}.h5')

In [ ]:
new_model.summary()